In [ ]:
import requests
import pandas as pd

def fetch_foods_by_categories(categories, page_size=100, max_pages=10):
    """Fetch food data for specific categories from Open Food Facts API."""
    base_url = "https://world.openfoodfacts.org/cgi/search.pl"
    foods = []
    
    for category in categories:
        print(f"Fetching data for category: {category}")
        for page in range(1, max_pages + 1):
            params = {
                "action": "process",
                "json": 1,
                "tagtype_0": "categories",
                "tag_contains_0": "contains",
                "tag_0": category,
                "page_size": page_size,
                "page": page,
            }
            response = requests.get(base_url, params=params)
            if response.status_code == 200:
                data = response.json()
                if "products" in data:
                    foods.extend(data["products"])
                else:
                    break
            else:
                print(f"Failed to fetch data for category: {category}, page {page}. Status Code: {response.status_code}")
                break
    return foods

def process_food_data(foods):
    """Extract relevant fields from the food data and return a cleaned DataFrame."""
    processed_foods = []
    for food in foods:
        processed_foods.append({
            "Product Name": food.get("product_name", None),
            "Brand": food.get("brands", None),
            "Category": food.get("categories", "N/A"),
            "Nutri-Score": food.get("nutriscore_grade", "N/A"),
            "Eco-Score": food.get("ecoscore_grade", "N/A"),
            "Energy (kcal)": food.get("nutriments", {}).get("energy-kcal_100g", "N/A"),
            "Sugars (g)": food.get("nutriments", {}).get("sugars_100g", "N/A"),
            "Saturated Fat (g)": food.get("nutriments", {}).get("saturated-fat_100g", "N/A"),
            "Salt (g)": food.get("nutriments", {}).get("salt_100g", "N/A"),
            "Fiber (g)": food.get("nutriments", {}).get("fiber_100g", "N/A"),
            "Proteins (g)": food.get("nutriments", {}).get("proteins_100g", "N/A"),
            "Fruits/Vegetables (%)": food.get("nutriments", {}).get("fruits-vegetables-nuts-estimate-from-ingredients_100g", "N/A"),
        })

    df = pd.DataFrame(processed_foods)
    df = df.dropna(subset=["Product Name"])
    df = df.dropna(subset=["Brand"])
    return df 

def create_foods_database(file_name="foods_france_filtered.xlsx", page_size=100, max_pages=10):
    """Create a database of foods for specified categories and save to an Excel file."""
    categories = [
        "Breakfast cereals",
        "Snacks",
        "Cookies",
        "Industrial ready-made dishes",
        "Beverages",
        "Dairy products"
    ]
    
    print("Fetching food data for specified categories...")
    foods = fetch_foods_by_categories(categories, page_size, max_pages)
    print(f"Fetched {len(foods)} foods.")
    
    print("Processing food data...")
    df = process_food_data(foods)
    
    print(f"Rows after cleaning: {len(df)}")
    
    print("Saving to Excel file...")
    df.to_excel(file_name, index=False) 
    print(f"Database saved as {file_name}.")

create_foods_database()


Fetching food data for specified categories...
Fetching data for category: Breakfast cereals
Fetching data for category: Snacks
Fetching data for category: Cookies
Failed to fetch data for category: Cookies, page 1. Status Code: 429
Fetching data for category: Industrial ready-made dishes
Failed to fetch data for category: Industrial ready-made dishes, page 1. Status Code: 429
Fetching data for category: Beverages
Failed to fetch data for category: Beverages, page 1. Status Code: 429
Fetching data for category: Dairy products
Failed to fetch data for category: Dairy products, page 1. Status Code: 429
Fetched 2000 foods.
Processing food data...
Nutri-Score counts:
  b: 120 (b)
  c: 508 (c)
  a: 310 (a)
  d: 507 (d)
  e: 513 (e)
  unknown: 36 (unknown)
  not-applicable: 6 (not-applicable)
Eco-Score counts:
  e: 23 (e)
  d: 29 (d)
  b: 29 (b)
  a: 5 (a)
  c: 19 (c)
  unknown: 11 (unknown)
  a-plus: 1 (a-plus)
  f: 1 (f)
Rows after cleaning: 1986
Saving to Excel file...
Database saved as f